# Modeling & Evaluation -- EV Adoption Across Swiss Cantons

## Purpose
This notebook implements and evaluates the **machine learning models**
used to explain and predict electric vehicle adoption across Swiss cantons.

## Modeling Strategy
The following model families are considered:
- Linear models: OLS, Ridge, Lasso
- Tree-based models: Random Forest, Gradient Boosting
- Non-linear models: Kernel methods and neural networks

All models are tuned using cross-validation grouped by canton
to avoid information leakage across regions.

## Evaluation Metrics
- R²
- RMSE
- MAE

## Outputs
- Cross-validated performance tables
- Model comparison figures
- Feature importance and interpretability outputs (where applicable)

## Execution
This notebook is executed via `run_pipeline.py`
as part of the full project pipeline.


In [ ]:
from pathlib import Path

# --------------------------------------------------
# Robust project root detection (Models)
# --------------------------------------------------
cwd = Path.cwd().resolve()

# Case 1: launched from project root
if (cwd / "data" / "intermediate").exists():
    ROOT = cwd

# Case 2: launched from notebooks/
elif (cwd.parent / "data" / "intermediate").exists():
    ROOT = cwd.parent

else:
    raise FileNotFoundError(
        f"Cannot locate project root from cwd={cwd}. "
        "Expected 'data/intermediate/' in cwd or parent."
    )

DATA = ROOT / "data"
INTER = DATA / "intermediate"
OUT = DATA / "outputs"

OUT.mkdir(parents=True, exist_ok=True)

print("ROOT :", ROOT)
print("INTER:", INTER)
print("OUT  :", OUT)

In [ ]:
# ============================================================
# OUTPUT FOLDERS (FIGURES + TABLES)
# ============================================================
FIG_DIR = OUT / "figures"
TAB_DIR = OUT / "tables"
FIG_DIR.mkdir(parents=True, exist_ok=True)
TAB_DIR.mkdir(parents=True, exist_ok=True)

def save_fig(filename: str, dpi: int = 200):
    """Save current matplotlib figure to Outputs/figures/"""
    path = FIG_DIR / filename
    plt.tight_layout()
    plt.savefig(path, dpi=dpi, bbox_inches="tight")
    print(f" Figure saved: {path}")
    plt.close()

def save_table(df, filename_csv: str, filename_xlsx: str | None = None):
    """Save a DataFrame to Outputs/tables/ as CSV (+ optional XLSX)."""
    csv_path = TAB_DIR / filename_csv
    df.to_csv(csv_path, index=False)
    print(f" Table saved: {csv_path}")
    if filename_xlsx:
        xlsx_path = TAB_DIR / filename_xlsx
        df.to_excel(xlsx_path, index=False)   
        print(f" Table saved: {xlsx_path}")


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import statsmodels.api as sm

# Imports for Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from scipy.stats import uniform, randint

# ============================================================
# 0. Setup and loading the data
# ============================================================
csv_path = INTER / "master_panel_2015_2024.csv"
assert csv_path.exists(), f"Missing file: {csv_path}"
df = pd.read_csv(csv_path)

if not csv_path.exists():
    raise FileNotFoundError(f"Can't find: {csv_path}")

print("Loading the data...")
df = pd.read_csv(csv_path)

# Initial setup
df_model = df.copy()
# Cleaning
df_model = df_model.dropna(subset=["canton_code", "ev_reg_share"]).reset_index(drop=True)

# Target variables and groups
df_model["ev_reg_share"] = df_model["ev_reg_share"].astype(float)
y = df_model["ev_reg_share"].values
df_model["trend"] = df_model["year"] - df_model["year"].min()
groups = df_model["canton_code"].values
gkf = GroupKFold(n_splits=5)

# Standardised assessment function
def evaluate_model(model, X, y, cv, groups, model_name):
    r2s, rmses, maes = [], [], []
    for train_idx, test_idx in cv.split(X, y, groups):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test  = scaler.transform(X_test)

        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        r2s.append(r2_score(y_test, preds))
        rmses.append(np.sqrt(mean_squared_error(y_test, preds)))
        maes.append(mean_absolute_error(y_test, preds))

    return {
        "Model": model_name,
        "R2": float(np.mean(r2s)),
        "RMSE": float(np.mean(rmses)),
        "MAE": float(np.mean(maes)),
    }

# ============================================================
# 1. CLUSTERING 
# ============================================================
print("Starting clustering (Strategy for saving linear models)...")
canton_profile = df_model.groupby("canton_code")[[
    "gdp_per_capita_chf", "pop_dec31", "motorization_rate_per_1000",
    "summer_temp_c", "co2_emissions_mt"
]].mean()

# Imputation to avoid KMeans crash
imputer = SimpleImputer(strategy="median")
canton_profile_clean = pd.DataFrame(
    imputer.fit_transform(canton_profile),
    columns=canton_profile.columns,
    index=canton_profile.index
)

# K-Means
scaler_cluster = StandardScaler()
X_cluster = scaler_cluster.fit_transform(canton_profile_clean)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
canton_profile["cluster_id"] = kmeans.fit_predict(X_cluster)

# Mapping of clusters
cluster_map = canton_profile["cluster_id"].to_dict()
df_model["canton_cluster"] = df_model["canton_code"].map(cluster_map)
cluster_dummies = pd.get_dummies(df_model["canton_cluster"], prefix="cluster", drop_first=True)

# ============================================================
# 2. FEATURE ENGINEERING & MATRIXES
# ============================================================
df_model = df_model.fillna(df_model.median(numeric_only=True))

df_model["income_vs_elec"] = df_model["gdp_per_capita_chf"] / (df_model["price_elcom_cts"] + 1)
df_model["mot_x_pop"] = df_model["motorization_rate_per_1000"] * np.log(df_model["pop_dec31"] + 1)

base_features = [
    "motorization_rate_per_1000", "gdp_per_capita_chf", "price_elcom_cts",
    "summer_temp_c", "winter_temp_c", "co2_emissions_mt", "greens_share",
    "pop_dec31", "income_vs_elec", "mot_x_pop"
]

# Linear Matrix (Features + CLUSTERS) -> For Ridge/Lasso
X_lin_df = pd.concat([df_model[base_features], cluster_dummies], axis=1)
X_lin = X_lin_df.values

# Non-Linear Matrix (Features + TREND) -> For Trees/GBR
X_nonlin_df = pd.concat([df_model[base_features], df_model[["trend"]]], axis=1)
nonlin_features = X_nonlin_df.columns.tolist() 
X_nonlin = X_nonlin_df.values

# ============================================================
# 3. MODEL TRAINING
# ============================================================
results = [] 
print("Training the models...")

# A. Linear Models (Ridge/Lasso handle multicollinearity)
results.append(evaluate_model(LinearRegression(), X_nonlin, y, gkf, groups, "OLS (Simple)"))
results.append(evaluate_model(RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0]), X_lin, y, gkf, groups, "Ridge CV (Clusters)"))
results.append(evaluate_model(LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1], max_iter=10000), X_lin, y, gkf, groups, "Lasso CV (Clusters)"))

# B. Non-Linear & Advanced Models
results.append(evaluate_model(RandomForestRegressor(n_estimators=300, random_state=42), X_nonlin, y, gkf, groups, "Random Forest"))

mlp = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', alpha=0.01, max_iter=1000, early_stopping=True, random_state=42)
results.append(evaluate_model(mlp, X_nonlin, y, gkf, groups, "Neural Net (MLP)"))

svr = SVR(kernel='rbf', C=10.0, epsilon=0.1)
results.append(evaluate_model(svr, X_nonlin, y, gkf, groups, "SVR (RBF Kernel)"))

kernel = ConstantKernel(1.0) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1e-3, noise_level_bounds=(1e-10, 1e+1))
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, random_state=42)
results.append(evaluate_model(gpr, X_nonlin, y, gkf, groups, "Gaussian Process"))

# C. Gradient Boosting Tuned 
print("Tuning Gradient Boosting (advanced optimization)...")
param_dist = {
    "n_estimators": randint(200, 1000),         # More trees
    "learning_rate": uniform(0.01, 0.2),
    "max_depth": randint(2, 5),                 # Shallow trees to avoid overfitting
    "subsample": uniform(0.6, 0.4),             # Stochastic Gradient Boosting
    "min_samples_split": randint(2, 10),
    "min_samples_leaf": randint(2, 10),
    "max_features": ["sqrt", "log2", None],     # Randomization of features
    "n_iter_no_change": randint(10, 20),        # Early Stopping (stops if no amelioration)
    "validation_fraction": uniform(0.1, 0.1)    # Data for the early stopping
}

gbr = GradientBoostingRegressor(random_state=42)
search = RandomizedSearchCV(gbr, param_dist, n_iter=25, cv=gkf, scoring="r2", n_jobs=-1, verbose=0, random_state=42)
search.fit(X_nonlin, y, groups=groups)
best_model = search.best_estimator_

results.append(evaluate_model(best_model, X_nonlin, y, gkf, groups, "Gradient Boosting (Tuned)"))

# ============================================================
# 4. EXPORT
# ============================================================
df_models = pd.DataFrame(results)
order = ["OLS (Simple)", "Ridge CV (Clusters)", "Lasso CV (Clusters)", "Random Forest", 
         "Neural Net (MLP)", "SVR (RBF Kernel)", "Gaussian Process", "Gradient Boosting (Tuned)"]
df_models["order"] = df_models["Model"].apply(lambda x: order.index(x) if x in order else 99)
df_models = df_models.sort_values("order").drop(columns="order")

print("\n=== TABLE OF RESULTS ===")
print(df_models)
save_table(df_models, "Model results.csv", "Model results.xlsx")

# ============================================================
# 5. SHAP & OLS (Diagnostic)
# ============================================================
print("\nGenerating the SHAP graph...")
best_model.fit(X_nonlin, y)
X_analysis = pd.DataFrame(X_nonlin, columns=nonlin_features)
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_analysis)

plt.figure(figsize=(10, 6))
plt.title("Factors (SHAP)")
shap.summary_plot(shap_values, X_analysis, show=False)
save_fig("SHAP Graph.png", dpi=300)

print("\n=== OLS Statistical analysis (Multicollinearity & P-values) ===")
X_sm = sm.add_constant(X_analysis)
model_sm = sm.OLS(y, X_sm).fit()
print(model_sm.summary())


In [ ]:
# ============================================================
# 5. FINAL TABLE & EXPORT
# ============================================================

# Transforming the results list into a DataFrame
df_models = pd.DataFrame(results)

# Logical order for the report (from simplest to most complex/new)
order = [
    "OLS (Simple)",              
    "Ridge CV (Clusters)",       
    "Lasso CV (Clusters)",       
    "Random Forest",             
    "Gradient Boosting (Tuned)", 
    "Neural Net (MLP)",          
    "SVR (RBF Kernel)",          
    "Gaussian Process"           
]

# Applying custom sorting
# (We handle cases where a model is missing with a default value of 99)
df_models["order"] = df_models["Model"].apply(lambda x: order.index(x) if x in order else 99)
df_models = df_models.sort_values("order").drop(columns="order")

print("\n=== FINAL TABLE — all models ===")
print(
    df_models.to_string(
        index=False,
        formatters={
            "R2":   "{:.3f}".format,
            "RMSE": "{:.3f}".format,
            "MAE":  "{:.3f}".format,
        },
    )
)

export_path = OUT / "model_results_final.csv"
df_models.to_csv(export_path, index=False)

print(f"\n File exported : {export_path}")

In [ ]:
# ============================================================
# 10. ECONOMETRIC DIAGNOSTICS 
# ============================================================
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro
import pandas as pd
import numpy as np

print("Start of residue analysis on the OLS model...")

# 1. Preparing data for Statsmodels
# We use X_nonlin because it contains all the relevant features (including the trend).
try:
    # We retrieve the column names defined previously.
    cols = nonlin_features
except NameError:
    # Security fallback if the variable does not exist
    cols = [f"Var_{i}" for i in range(X_nonlin.shape[1])]

X_sm_df = pd.DataFrame(X_nonlin, columns=cols)
X_sm = sm.add_constant(X_sm_df)  # Adding the constant (intercept)

# 2. Fit of the complete OLS model on the entire dataset
model_sm = sm.OLS(y, X_sm).fit()

# .values.flatten() ensures that these are simple 1D arrays
# This avoids "Multi-dimensional indexing" errors with matplotlib/seaborn.
residuals = model_sm.resid.values.flatten()
y_pred = model_sm.fittedvalues.values.flatten()

# --- A. NORMALITY TEST ---
plt.figure(figsize=(12, 5))

# Histogram
plt.subplot(1, 2, 1)
plt.hist(residuals, bins=20, density=True, color='skyblue', edgecolor='black', alpha=0.7)
plt.title("Distribution of residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")

# QQ-Plot
plt.subplot(1, 2, 2)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Normal Q–Q plot of OLS residuals")
plt.tight_layout()
save_fig("OLS Residuals Distribution_qq.png")

# Shapiro-Wilk Test
stat, p_shapiro = shapiro(residuals)
print(f"Shapiro-Wilk test : p-value = {p_shapiro:.4e}")
if p_shapiro > 0.05:
    print("H0 accepted: Residuals are normally distributed.")
else:
    print("H0 rejected: Residuals do not follow a perfect normal distribution.")

# --- B. HOMOSCEDASTICITY TEST (Breusch-Pagan) ---
plt.figure(figsize=(8, 5))
plt.scatter(y_pred, residuals, alpha=0.6, edgecolors='b')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("fitted values")
plt.ylabel("Residuals")
plt.title("Residuals versus fitted values for the OLS model (Homoscedasticity test)")
save_fig("OLS Residuals vs Fitted.png")

# Breusch-Pagan Test
bp_test = het_breuschpagan(model_sm.resid, model_sm.model.exog)
print(f"Breusch-Pagan test: p-value = {bp_test[1]:.4e}")
if bp_test[1] > 0.05:
    print("Confirmed homoscedasticity (constant variance).")
else:
    print("Heteroscedasticity detected (The variance of errors changes).")

# --- C. INDEPENDENCE AND INFLUENCE (Cook's Distance) ---
influence = model_sm.get_influence()
resid_std = influence.resid_studentized_internal

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(resid_std, marker='o', linestyle='', alpha=0.5)
plt.axhline(2, color='red', linestyle='--')
plt.axhline(-2, color='red', linestyle='--')
plt.title("Standardized Residuals (Outliers if > 2)")

# Cook's Distance
cooks = influence.cooks_distance[0]

plt.subplot(1, 2, 2)
plt.stem(np.arange(len(cooks)), cooks, basefmt=" ")
plt.title("Cook’s distance for the OLS model")
plt.xlabel("Index Observation")
save_fig("ols_outliers_cooks_distance.png")

threshold = 4 / len(y)
influential_points = np.where(cooks > threshold)[0]
print(f"Number of influential observations (Cook's D > {threshold:.3f}) : {len(influential_points)}")

# --- Statistical summary ---
print("\n" + "="*40)
print("STATISTICAL SUMMARY OF THE OLS MODEL")
print("="*40)
print(model_sm.summary())
summary_path = TAB_DIR / "OLS Diagnostic Summary.txt"
with open(summary_path, "w", encoding="utf-8") as f:
    f.write(model_sm.summary().as_text())
print(f"OLS summary saved: {summary_path}")

In [ ]:
vars_desc = [c for c in df_model.columns if c in [
    "y",
    "price_elcom_cts",
    "gdp_per_capita_chf",
    "motorization_rate_per_1000",
    "pop_dec31",
    "summer_temp_c",
    "winter_temp_c",
    "co2_emissions_mt",
    "greens_share"
]]

print("Variables used :", vars_desc)

desc_table = (
    df_model[vars_desc]
    .describe()
    .loc[["count", "mean", "std", "min", "max"]]
    .T
)

desc_table
